In [1]:
import networkx as nx
from itertools import combinations

def build_cooccurrence_network(play):
    G = nx.Graph()
    
    for act in play["acts"]:
        for scene in act["scenes"]:
            speakers = scene["speakers_in_scene"]
            for s in speakers:
                G.add_node(s)
            for a, b in combinations(speakers, 2):
                if G.has_edge(a, b):
                    G[a][b]["weight"] += 1
                else:
                    G.add_edge(a, b, weight=1)
    return G


In [ ]:
import networkx as nx

def speech_length(text):
    # veldig enkel: antall ord
    return len(text.split())

def build_speech_network_and_transitions(play):
    G = nx.DiGraph()
    transitions = []  # rad per overgang A->B

    for act in play["acts"]:
        for scene in act["scenes"]:
            speakers = scene["speakers_in_scene"]
            speeches = scene["speeches"]
            if len(speeches) < 2:
                continue

            # precompute lengths
            seq = []
            for sp in speeches:
                L = speech_length(sp["text"])
                seq.append({
                    "speaker": sp["speaker"],
                    "length": L
                })

            # gå gjennom sekvensen
            for i in range(len(seq) - 1):
                cur = seq[i]
                nxt = seq[i+1]
                a = cur["speaker"]
                b = nxt["speaker"]
                len_a = cur["length"]
                len_b = nxt["length"]

                # legg til i graf
                if not G.has_node(a):
                    G.add_node(a)
                if not G.has_node(b):
                    G.add_node(b)
                if G.has_edge(a, b):
                    G[a][b]["count"] += 1
                    G[a][b]["len_A_sum"] += len_a
                    G[a][b]["len_B_sum"] += len_b
                else:
                    G.add_edge(a, b,
                               count=1,
                               len_A_sum=len_a,
                               len_B_sum=len_b)

                # lag en «rad» for senere modellering
                transitions.append({
                    "play": play.get("title", ""),
                    "act": act["act_n"],
                    "scene": scene["scene_n"],
                    "current_speaker": a,
                    "next_speaker": b,
                    "len_current": len_a,
                    "len_next": len_b,
                    # hele scenens cast – viktig for «C tilstede?»-spørsmål
                    "scene_speakers": tuple(sorted(speakers)),
                })

    return G, transitions


In [7]:
import re
from lxml import etree

NS = {
    "tei": "http://www.tei-c.org/ns/1.0",
    "HIS": "http://www.example.org/ns/HIS",
}
# XML_NS trengs ikke lenger hvis vi bytter navn
# XML_NS = "{http://www.w3.org/XML/1998/namespace}"

def tokenize(text):
    return re.findall(r"\w+|\S", text, flags=re.UNICODE)

def speech_length(text):
    return len(tokenize(text))

def parse_play_xml(xml_text, title="", filename=""):
    # 🔧 1) Nøytraliser xml:id → xml_id (slipper NCName-reglene)
    xml_text = xml_text.replace("xml:id", "xml_id")

    # 2) Parse
    root = etree.fromstring(xml_text.encode("utf-8"))

    # 3) cast list / role mapping
    char_map = {}
    for item in root.xpath("//tei:castItem", namespaces=NS):
        xml_id = item.get("xml_id")               # ⬅️ endret her
        name = " ".join(item.xpath(".//text()")).strip()
        if xml_id and name:
            char_map[xml_id] = name
            char_map["#" + xml_id] = name

    def extract_sp(sp):
        # fjern stage directions
        for stage in sp.xpath(".//HIS:hisStage", namespaces=NS):
            stage.getparent().remove(stage)
        txt = sp.xpath("string()").strip()
        who_raw = sp.get("who")
        speaker = char_map.get(who_raw, who_raw)
        return {
            "speaker": speaker,
            "text": txt,
            "length": speech_length(txt),
        }

    acts_out = []
    acts = root.xpath("//tei:div[@type='act']", namespaces=NS)
    if not acts:
        acts = [root]

    for act in acts:
        act_n = act.get("n", "")
        scenes = act.xpath("./tei:div[@type='scene']", namespaces=NS)
        if not scenes:
            scenes = [act]

        scenes_out = []
        for sc in scenes:
            scene_n = sc.get("n", "")
            speeches = [extract_sp(sp) for sp in sc.xpath(".//HIS:hisSp", namespaces=NS)]
            speakers = sorted({s["speaker"] for s in speeches if s["speaker"]})
            scenes_out.append({
                "scene_n": scene_n,
                "speakers_in_scene": speakers,
                "speeches": speeches,
            })

        acts_out.append({"act_n": act_n, "scenes": scenes_out})

    return {
        "title": title,
        "file": filename,
        "acts": acts_out,
    }


In [6]:
import glob, json

all_plays = []
for fn in glob.glob("plays/*.xml"):
    with open(fn, encoding="utf-8") as f:
        txt = f.read()

    title = fn.split("/")[-1].replace(".xml", "")
    data = parse_play_xml(txt, title=title, filename=fn)
    all_plays.append(data)

with open("ibsen_parsed.json", "w", encoding="utf-8") as f:
    json.dump(all_plays, f, ensure_ascii=False, indent=2)
